In [424]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math

from time import time
import logging
import pylab as pl

from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from  sklearn.neural_network import MLPRegressor

In [340]:
train = pd.read_csv('data/trip_train.csv')

In [341]:
test = pd.read_csv('data/trip_test.csv')

In [342]:
weather = pd.read_csv('data/weather.csv')

In [343]:
station = pd.read_csv('data/station.csv')

In [344]:
#Preprocesing

In [345]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationProcesed = stationReduced.drop(labels=['name','city'],axis=1)


In [346]:
stationProcesed.head()

,id,zip_code
0,2,95113
1,3,95113
2,4,95113
3,5,95113
4,6,95113


In [347]:
test = test.drop(labels=['zip_code',\
                         'end_date',\
                         'end_station_name',\
                         'end_station_id',\
                         'bike_id'],\
                 axis=1)

In [348]:
train = train.drop(labels=['zip_code',\
                           'end_date',\
                           'end_station_name',\
                           'end_station_id',\
                           'bike_id'],\
                   axis=1)

In [349]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [350]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [351]:
weather.dtypes

date                               object
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    float64
min_sea_level_pressure_inches     float64
max_visibility_miles              float64
mean_visibility_miles             float64
min_visibility_miles              float64
max_wind_Speed_mph                float64
mean_wind_speed_mph               float64
max_gust_speed_mph                float64
precipitation_inches              float64
cloud_cover                       float64
events                             object
wind_dir_degrees                  float64
zip_code                          

In [352]:
# PreProcessing trips

In [353]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [354]:
train['time'] = train['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )
test['time'] = test['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [355]:
train.loc[:,'start_date'] = train['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))
test.loc[:,'start_date'] = test['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))


In [356]:
train.loc[:,'day_of_week'] = train['start_date'].apply(lambda x : x.dayofweek )
test.loc[:,'day_of_week'] = test['start_date'].apply(lambda x : x.dayofweek )

In [357]:
train.loc[:,'month'] = train['start_date'].apply(lambda x : x.month )
test.loc[:,'month'] = test['start_date'].apply(lambda x : x.month )

In [358]:
train.loc[:,'year'] = train['start_date'].apply(lambda x : x.year )
test.loc[:,'year'] = test['start_date'].apply(lambda x : x.year )

In [359]:
train.loc[:,'dayofyear'] = train['start_date'].apply(lambda x : x.dayofyear )
test.loc[:,'dayofyear'] = test['start_date'].apply(lambda x : x.dayofyear )

In [360]:
train.loc[:,'start_date'] = train['start_date'].apply(lambda x : pd.to_datetime(x).date())
test.loc[:,'start_date'] = test['start_date'].apply(lambda x : pd.to_datetime(x).date())


In [361]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [362]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [363]:
train = pd.merge(holiday_festive_day,train,on=['start_date'],how='right')
test = pd.merge(holiday_festive_day,test,on=['start_date'],how='right')
test.shape

(119998, 11)

In [364]:
train.loc[:,'is_holiday'] = train.holiday.isnull()
test.loc[:,'is_holiday'] = test.holiday.isnull()

In [365]:
train.loc[:,'is_holiday'] = train['is_holiday'].apply(lambda x : not x )
test.loc[:,'is_holiday'] = test['is_holiday'].apply(lambda x : not x )
test.shape

(119998, 12)

In [366]:
train.loc[:,'subscription_type'] = train['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
test.loc[:,'subscription_type'] = test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
test.shape

(119998, 12)

In [367]:
#Merging dataframes

In [368]:
stationProcesed.rename(columns={'id':'start_station_id'},inplace=True)


In [369]:
train2 = pd.merge(train,\
                  stationProcesed,\
                  on= ['start_station_id'],\
                  how='inner')

test2 = pd.merge(test,\
                  stationProcesed,\
                  on= ['start_station_id'],\
                  how='inner')


In [370]:
# Preprocessing trips

In [371]:
train2 = train2.drop(labels=['holiday'],axis=1).dropna()

test2 = test2.drop(labels=['holiday'],axis=1).dropna()

In [372]:
train2.rename(columns={'start_date':'date'},inplace=True)
test2.rename(columns={'start_date':'date'},inplace=True)

In [373]:
# merging with weather, trainN where N - 1 count merges

In [374]:
train3 = pd.merge(train2,weather,\
                  on=['date','zip_code'],\
                  how='left')

test3 = pd.merge(test2,weather,\
                 on=['date','zip_code'],\
                 how='left')

In [375]:
train3.rename(columns={'start_station_id':'station_id'},inplace=True)

test3.rename(columns={'start_station_id':'station_id'},inplace=True)

In [376]:
# register with nan values in weather

In [377]:
nan_rows_test = test3[test3.isnull().T.any().T]
nan_rows_test.shape# 3 registers


(3, 34)

In [378]:
train3
nan_rows_train = train3[train3.isnull().T.any().T]
nan_rows_train#11 registers

,date,id,duration,start_station_name,station_id,subscription_type,time,day_of_week,month,year,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees
480775,2015-08-08,881327,4398,University and Emerson,35,1,1048,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480776,2015-08-08,881430,8274,University and Emerson,35,1,1250,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480777,2015-08-08,881675,2574,University and Emerson,35,1,1829,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480778,2015-08-08,881487,3581,University and Emerson,35,1,1404,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480779,2015-08-08,881429,8274,University and Emerson,35,1,1251,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480780,2015-08-08,881331,4289,University and Emerson,35,1,1049,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480781,2015-08-08,881326,4435,University and Emerson,35,1,1047,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531946,2015-08-08,881451,1755,Cowper at University,37,1,1323,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531947,2015-08-08,881341,3123,Cowper at University,37,1,1103,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
533503,2015-08-08,881512,6104,Palo Alto Caltrain Station,34,1,1435,5,8,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [379]:
# zeros in nan values'test3

In [380]:
train3 = train3.dropna()
test3 = test3.fillna(0)

In [381]:
test3.shape

(119998, 34)

In [382]:
train3.shape

(549950, 35)

In [383]:
# ML robert


In [275]:
train3.dtypes

date                               object
id                                  int64
duration                            int64
start_station_name                 object
station_id                          int64
subscription_type                   int64
time                                int64
day_of_week                         int64
month                               int64
year                                int64
dayofyear                           int64
is_holiday                           bool
zip_code                            int64
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    

In [384]:
import sklearn.preprocessing
def numberEvents(df):
    df['events'] = df.events.apply(eventNumber)

def eventNumber(eventString):
    if eventString == 'Normal' or eventString == 0: return 0
    elif eventString == 'Rain': return 1
    elif eventString == 'Fog': return 2
    elif eventString == 'Fog-Rain': return 3
    else: return 4
    
def cleanEvents(df):
    df['events'] = df.events.apply(lambda x: "Normal" if pd.isnull(x) else x)
    df['events'] = df.events.apply(lambda x: "Rain" if x == "rain" else x)

def encode(df):
    encoder = sklearn.preprocessing.OneHotEncoder()
    return pd.DataFrame(encoder.fit_transform(df).toarray())

In [385]:
trainCategories = train3.loc[:,['station_id',\
                        'subscription_type',
                       'day_of_week',\
                       'year',\
                       'month',\
                       'is_holiday',
                        'events'\
                       ]]
testCategories = test3.loc[:,['station_id',\
                        'subscription_type',
                       'day_of_week',\
                       'year',\
                       'month',\
                       'is_holiday',
                        'events'\
                       ]]

In [386]:
numberEvents(testCategories)

In [387]:
numberEvents(trainCategories)

In [388]:
trainCategories = encode(trainCategories)

In [389]:
testCategories = encode(testCategories)

In [390]:
testNoCategories = test3.iloc[:,12:]
del testNoCategories['events']


In [391]:
trainNoCategories = train3.iloc[:,13:]
del trainNoCategories ['events']
trainNoCategories['duration'] = train3['duration']

In [392]:
trainR = trainCategories.join(trainNoCategories, lsuffix = 'x',rsuffix='y')


In [393]:
testR = testCategories.join(testNoCategories, lsuffix = 'x',rsuffix='y')

In [396]:
trainR.dtypes

0                                 float64
1                                 float64
2                                 float64
3                                 float64
4                                 float64
5                                 float64
6                                 float64
7                                 float64
8                                 float64
9                                 float64
10                                float64
11                                float64
12                                float64
13                                float64
14                                float64
15                                float64
16                                float64
17                                float64
18                                float64
19                                float64
20                                float64
21                                float64
22                                float64
23                                

In [399]:
trainR = trainR.loc[lambda x: x['duration'] < 10800,:]

In [402]:
train_target = trainR['duration']

In [401]:
trainR.isnull().sum().sum()

0

In [286]:
trainR = trainR.dropna()
trainR.shape

(549939, 123)

In [287]:
train_target.shape

(549939,)

In [288]:
trainTarjet = trainR['duration']
del trainR['duration']

In [407]:
trainR.iloc[:,101:]
del trainR['duration']

In [405]:
testR.iloc[:,101:]

,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_inches,...,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees
0,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,29.97,...,29.90,10.0,10.0,6.0,23.0,12.0,30.0,0.000,6.0,277.0
1,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,29.97,...,29.90,10.0,10.0,6.0,23.0,12.0,30.0,0.000,6.0,277.0
2,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,29.97,...,29.90,10.0,10.0,6.0,23.0,12.0,30.0,0.000,6.0,277.0
3,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
4,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
5,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
6,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
7,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
8,77.0,64.0,51.0,50.0,41.0,36.0,71.0,49.0,27.0,30.12,...,30.01,10.0,10.0,10.0,18.0,5.0,0.0,0.000,0.0,272.0
9,62.0,58.0,54.0,51.0,49.0,46.0,83.0,72.0,60.0,30.17,...,30.09,10.0,9.0,6.0,12.0,5.0,15.0,0.000,7.0,76.0


In [408]:
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 5



pca = RandomizedPCA(n_components=n_components, whiten=True).fit(trainR.iloc[:,101:])



X_train_pca = pca.transform(trainR.iloc[:,101:])
X_test_pca = pca.transform(testR.iloc[:,101:])

/Users/Robert/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


In [409]:
trainPca = pd.DataFrame(X_train_pca)
testPca = pd.DataFrame(X_test_pca)

In [410]:
trainR = trainR.iloc[:,:100].join(trainPca, lsuffix = 'x',rsuffix='y')

In [411]:
testR = testR.iloc[:,:100].join(testPca, lsuffix = 'x',rsuffix='y')

In [172]:
rfr = ExtraTreesRegressor(n_jobs = -1,\
                         n_estimators=50,criterion='mse',oob_score=True,\
                         bootstrap=True)

In [173]:
rfr.fit(trainR,train_target)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [161]:
testR.shape

(119998, 122)

In [174]:
test_prediction = rfr.predict(testR)

In [175]:
x = pd.read_csv('data/trip.csv',
                   usecols=['id','duration'])
target = pd.read_csv('data/trip_test.csv',
                   usecols=[ 'id'])
testTarget = pd.merge(x,target, how = "inner", on = "id")

In [176]:
test_prediction.shape

(119998,)

In [177]:
testTarget.shape

(119998, 2)

In [178]:
mean_squared_error(testTarget['duration'], test_prediction )

71316160.722631335

In [ ]:
42923231
78019880
65412596

In [253]:
# ML SVR

In [255]:
from sklearn.svm import SVR

In [258]:
#svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
#svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
#y_rbf = svr_rbf.fit(X, y).predict(X)
#y_lin = svr_lin.fit(X, y).predict(X)


In [ ]:
# ML MLPRegressor

In [186]:
mlp = MLPRegressor()

In [192]:
trainR.head()

,0,1,2,3,4,5,6,7,8,9,...,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.9,10.0,10.0,6.0,23.0,12.0,30.0,0.0,6.0,277.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.9,10.0,10.0,6.0,23.0,12.0,30.0,0.0,6.0,277.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.9,10.0,10.0,6.0,23.0,12.0,30.0,0.0,6.0,277.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.9,10.0,10.0,6.0,23.0,12.0,30.0,0.0,6.0,277.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.9,10.0,10.0,6.0,23.0,12.0,30.0,0.0,6.0,277.0


In [426]:
trainR.iloc[:,:100]

,0x,1x,2x,3x,4x,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
mlp.fit(trainR.iloc[:,:100], train_target)

In [190]:
pred1 = mlp.predict(testR.iloc[:,:100])

In [191]:
mean_squared_error(testTarget['duration'], pred1)

43860624.42173028

In [290]:
pca = IncrementalPCA(n_components=8)

In [291]:
#trainR.iloc[:,:101]
testR.iloc[:,:101]

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [292]:
tripTransform = pca.fit_transform(trainR.iloc[:,101:])
tripTransform_test = pca.fit_transform(testR.iloc[:,101:])

In [293]:
tripTransformDF = pd.DataFrame(tripTransform)
tripTransformDF_test = pd.DataFrame(tripTransform_test)

In [294]:
trainRpca = tripTransformDF
testRpca = tripTransformDF_test

In [301]:
trainRpca.shape

(549939, 8)

In [304]:
trainRpca.head()

,0,1,2,3,4,5,6,7
0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
1,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
2,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
3,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
4,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945


In [302]:
testRpca.shape

(119998, 8)

In [295]:
trainR = trainR.iloc[:,:101].join(trainRpca, lsuffix = 'x',rsuffix='y')
testR = testR.iloc[:,:101].join(testRpca, lsuffix = 'x',rsuffix='y')

In [323]:
print(trainR.shape, trainR.isnull().values.any())
print(train_target.shape, train_target.isnull().values.any())

((549939, 109), True)
((549939,), False)


In [415]:
nonNulls = trainR.notnull().all(axis=1)
trainR2 = trainR[nonNulls]
train_target2 = train_target[nonNulls]

In [416]:
print(trainR2.shape, trainR2.isnull().values.any())
print(train_target2.shape, train_target2.isnull().values.any())

((536367, 105), False)
((536367,), False)


In [305]:
trainR.head()

,0x,1x,2x,3x,4x,5x,6x,7x,8,9,...,99,100,0y,1y,2y,3y,4y,5y,6y,7y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18.30496,22.690649,16.535694,2.357739,2.996042,1.119105,-0.758394,1.593945


In [326]:

nan_rows_train = trainR2[trainR2.isnull().T.any().T]
nan_rows_train

,0x,1x,2x,3x,4x,5x,6x,7x,8,9,...,99,100,0y,1y,2y,3y,4y,5y,6y,7y


In [337]:
trainR2.shape

(549928, 109)

In [338]:
train3.shape

(549950, 35)

In [303]:
trainR.iloc[:,:101].shape

(549939, 101)

In [412]:
trainR.shape

(543133, 105)

In [417]:
mlp.fit(trainR2, train_target2)

/Users/Robert/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [419]:
pred1 = mlp.predict(testR)

In [422]:
testR.shape

(119998, 105)

In [423]:
trainR.shape

(543133, 105)

In [420]:
mean_squared_error(testTarget['duration'], pred1  )

856048.40661407472

In [421]:
testTarget.to_csv('data/prediction_4_kaggle.csv', index=False)

In [330]:
mean_squared_error(testTarget['duration'], pred1  )

1466136.5121480385

In [331]:
testTarget['duration'] = pred1

In [332]:
testTarget.head()

,id,duration
0,4130,2756.079976
1,4498,2756.079976
2,4557,2756.079976
3,4386,454.239521
4,4242,454.239521


In [333]:
testTarget.to_csv('data/prediction_3_kaggle.csv', index=False)

In [93]:
# probando el ML de braian


In [116]:
trainBR = train3[train3['duration'] < 5000]
trainBR.shape

(536049, 35)

In [95]:
test_id = test3['id']

In [117]:
train_target = trainBR['duration']

In [100]:
trainBR= trainBR.drop(labels=['duration','date','events','id'],axis=1)

In [101]:
testBR = test3.drop(labels=['date','events','id'],axis=1)

In [102]:
from sklearn.decomposition import IncrementalPCA

In [103]:
pca = IncrementalPCA(n_components=8)

In [106]:

del trainBR['start_station_name']
del testBR['start_station_name']

In [107]:
tripTransform = pca.fit_transform(trainBR)
tripTransform_test = pca.fit_transform(testBR)

In [108]:
tripTransform.shape

(536049, 8)

In [109]:
tripTransform_test.shape

(119998, 8)

In [110]:
tripTransformDF = pd.DataFrame(tripTransform)
tripTransformDF_test = pd.DataFrame(tripTransform_test)

In [121]:
trainBR = tripTransformDF
testBR = tripTransformDF_test

In [112]:
from sklearn import ensemble

In [113]:
rfr = ensemble.RandomForestRegressor(n_jobs=-1,n_estimators=50,criterion='mse',oob_score=True)

In [122]:
train_target.shape

(536049,)

In [123]:
trainBR.shape

(536049, 8)

In [124]:
rfr.fit(trainBR,train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [125]:
test_prediction = rfr.predict(testBR)

In [126]:
x = pd.read_csv('data/trip.csv',
                   usecols=['id','duration'])
target = pd.read_csv('data/trip_test.csv',
                   usecols=[ 'id'])
testTarget = pd.merge(x,target, how = "inner", on = "id")

In [127]:
test_prediction.shape

(119998,)

In [128]:
testTarget.shape

(119998, 2)

In [129]:
testTarget.dtypes

id          int64
duration    int64
dtype: object

In [130]:
from sklearn.metrics import mean_squared_error

In [132]:
mean_squared_error(testTarget['duration'], test_prediction )

42923231.095528767

In [ ]:
42910497

In [133]:
testTarget['duration'] = test_prediction

In [135]:
testTarget.to_csv('data/prediction_1_kaggle.csv', index=False)